In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, concatenate, Add, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_Augmented']
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 8

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    for category in categories:
        class_num = categories.index(category)
        clear_path = os.path.join(base_dir, subfolders[0], category)
        clouds_path = os.path.join(base_dir, subfolders[1], category)
        clear_images = sorted(os.listdir(clear_path))
        clouds_images = sorted(os.listdir(clouds_path))
        
        for clear_img_name, clouds_img_name in zip(clear_images, clouds_images):
            if clear_img_name.endswith('.png') and clouds_img_name.endswith('.png'):
                clear_img_path = os.path.join(clear_path, clear_img_name)
                clouds_img_path = os.path.join(clouds_path, clouds_img_name)
                
                clear_img = tf.keras.preprocessing.image.load_img(clear_img_path, target_size=(img_height, img_width))
                clouds_img = tf.keras.preprocessing.image.load_img(clouds_img_path, target_size=(img_height, img_width))
                
                clear_img_array = tf.keras.preprocessing.image.img_to_array(clear_img)
                clouds_img_array = tf.keras.preprocessing.image.img_to_array(clouds_img)
                
                combined_img = np.concatenate((clear_img_array, clouds_img_array), axis=-1)
                
                data.append(combined_img)
                labels.append(class_num)
    return np.array(data), np.array(labels)

data, labels = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)

# Normalize the images
data = data / 255.0

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")

# Define data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply data augmentation to the training data
datagen.fit(X_train)

# Apply data augmentation to the training data
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

def residual_block(x, filters):
    shortcut = x
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_complex_model(input_shape):
    # Input layer for combined images
    combined_input = Input(shape=(input_shape[1], input_shape[2], input_shape[3]), name='combined_input')
    
    # Convolutional base
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(combined_input)
    x = MaxPooling2D((2, 2))(x)
    x = residual_block(x, 64)
    x = MaxPooling2D((2, 2))(x)
    x = residual_block(x, 128)
    x = MaxPooling2D((2, 2))(x)
    x = residual_block(x, 256)
    x = MaxPooling2D((2, 2))(x)
    x = residual_block(x, 512)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    
    # Fully connected layers
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=combined_input, outputs=output)
    return model

input_shape = X_train.shape
model = build_complex_model(input_shape)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Callbacks for training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (4832, 64, 64, 6)
Validation data shape: (1208, 64, 64, 6)
Training labels shape: (4832, 2)
Validation labels shape: (1208, 2)


C:\Anaconda\envs\thesis\lib\site-packages\keras\preprocessing\image.py:2094: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(
C:\Anaconda\envs\thesis\lib\site-packages\keras\preprocessing\image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 combined_input (InputLayer)    [(None, 64, 64, 6)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   1760        ['combined_input[0][0]']         
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 32, 32, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 64)   18496       ['max_pooling2d[0][0]']          
                                                                                              

In [ ]:
# TRAINING THE MODEL

# # Train the model with augmented data and class weights
# with tf.device('/GPU:0'):
#     history = model.fit(
#         train_generator,
#         steps_per_epoch=len(X_train) // BATCH_SIZE,
#         epochs=100,
#         validation_data=(X_val, y_val),
#         callbacks=[reduce_lr, early_stopping],
#         class_weight=class_weights
#     )

# # Make predictions
# with tf.device('/GPU:0'):
#     val_predictions = model.predict(X_val)

# # Convert one-hot encoded predictions and true labels to label indices
# y_val_true = np.argmax(y_val, axis=1)
# y_val_pred = np.argmax(val_predictions, axis=1)

# # Generate the confusion matrix
# conf_matrix = confusion_matrix(y_val_true, y_val_pred)

# print("Confusion Matrix:")
# print(conf_matrix)

# # Generate the classification report
# class_report = classification_report(y_val_true, y_val_pred, target_names=categories)

# print("Classification Report:")
# print(class_report)

In [12]:
# # Save the model
# model_save_path = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\succ_model_1\\my_model.h5'
# model.save(model_save_path)
# print(f"Model saved to {model_save_path}")

Model saved to C:\Users\Θάνος\Desktop\Thesis Thanasis\succ_model_1\my_model.h5


In [5]:
from tensorflow.keras.models import load_model

model_save_path = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\succ_model_1\\my_model.h5'

# Make predictions
with tf.device('/GPU:0'):
    val_predictions = model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
y_val_true = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(val_predictions, axis=1)

# Load the model
loaded_model = load_model(model_save_path)
print(f"Model loaded from {model_save_path}")

# Make predictions using the loaded model
with tf.device('/GPU:0'):
    loaded_val_predictions = loaded_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
loaded_y_val_pred = np.argmax(loaded_val_predictions, axis=1)

# Generate the confusion matrix
loaded_conf_matrix = confusion_matrix(y_val_true, loaded_y_val_pred)

print("Confusion Matrix:")
print(loaded_conf_matrix)

# Generate the classification report
loaded_class_report = classification_report(y_val_true, loaded_y_val_pred, target_names=categories)

print("Classification Report:")
print(loaded_class_report)

38/38 [==============================] - 2s 57ms/step
Model loaded from C:\Users\Θάνος\Desktop\Thesis Thanasis\succ_model_1\my_model.h5
38/38 [==============================] - 1s 30ms/step
Confusion Matrix:
[[572 152]
 [ 63 421]]
Classification Report:
                   precision    recall  f1-score   support

Healthy_augmented       0.90      0.79      0.84       724
Damaged_Augmented       0.73      0.87      0.80       484

         accuracy                           0.82      1208
        macro avg       0.82      0.83      0.82      1208
     weighted avg       0.83      0.82      0.82      1208

